# Cabinet Member Term Length & Approval vs. Spending

The goal here is to determine if there's any relationship between the vote confirmation ratio,
the member's term length, and their spending.

## Data pre-processing

Throw out unneeded columns, do some renaming to make things easier/pythonic...

In [27]:
import pandas as pd

cabinet = pd.read_csv("../data/csv/cabinet.csv")
print("Loaded {} cabinet member entries".format(len(cabinet)))
cabinet["Votes For"][cabinet["Vote Type"] == "Voice Vote"] = 95
cabinet["Votes Against"][cabinet["Vote Type"] == "Voice Vote"] = 5

cabinet: pd.DataFrame = cabinet.drop(cabinet.columns.difference(["Party", "Position", "Confirmed", "Votes For",
                                                                 "Votes Against"]), axis=1)\
    .rename({
        "Position": "position",
        "Party": "party",
        "Confirmed": "date",
        "Votes For": "votes_for",
        "Votes Against": "votes_against"
    }, axis=1)
cabinet.date = pd.to_datetime(cabinet.date)
cabinet.party = cabinet.party.map({"Republican": -1, "Democrat": 1})
cabinet["ratio"] = cabinet.votes_for / (cabinet.votes_for + cabinet.votes_against)


Loaded 87 cabinet member entries


<ipython-input-27-589b1efa6ea3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabinet["Votes For"][cabinet["Vote Type"] == "Voice Vote"] = 95
<ipython-input-27-589b1efa6ea3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabinet["Votes Against"][cabinet["Vote Type"] == "Voice Vote"] = 5
